## Introduction

I wrote a few Python scripts to help me with my blog. I didn't write them using Jupyter and nbdev, I used nvim, with Copilot to help me, and ChatGPT on the side.

I do have a way to edit Jupyer cells in nvim, though, using GhostText. This might be a good way to get the best of both worlds, Jupyter and vim and Copilot all together. I'll try that for the next scripts.

### blog_links

My Quarto blog is set up with a folder for each blog post, which can be a bit awkward. For example, the notebook for this post is at `blog/posts/automation/automation.ipynb`.

Also, the post folders are just short names, and don't indicate the order of the posts. So I decided to add numbered symlinks under the main `blog` directory to the notebook for each post.

In [ ]:
%cd ~/ai/blog
!ls -l 0*

/home/sam/ai/blog
lrwxrwxrwx 1 sam sam 21 Mar  6 16:27 001_blog.ipynb -> posts/blog/blog.ipynb
lrwxrwxrwx 1 sam sam 27 Mar  6 16:27 002_startup.ipynb -> posts/startup/startup.ipynb
lrwxrwxrwx 1 sam sam 31 Mar  6 16:27 003_shortcuts.ipynb -> posts/shortcuts/shortcuts.ipynb
lrwxrwxrwx 1 sam sam 23 Mar  6 16:27 004_setup.ipynb -> posts/setup/setup.ipynb
lrwxrwxrwx 1 sam sam 29 Mar  6 16:27 005_deadends.ipynb -> posts/deadends/deadends.ipynb
lrwxrwxrwx 1 sam sam 29 Mar  6 16:27 006_backport.ipynb -> posts/backport/backport.ipynb
lrwxrwxrwx 1 sam sam 33 Mar  6 17:31 007_automation.ipynb -> posts/automation/automation.ipynb


It was getting annoying to update these links by hand, so I wrote a Python script to help with it.

In [ ]:
#!/usr/bin/env python3

"""
This script creates numbered symlinks for blog posts.
"""

import os
from os.path import basename, islink
import re
from glob import glob
from pathlib import Path
import argparse
from os import environ as env

from lib import is_notebook, cd, show_locals, caller_locals, kv_dump, eprint
from blog import quarto_post_date

def update_links_in_cwd(digits=3, dry_run=False, verbose=False, debug=False):
	"""Update blog symlinks in the current working directory."""
	_params = caller_locals()

	# get list of blog symlinks
	blog_links = sorted(filter(islink, glob('*.ipynb')))

	# get list of blog posts under posts/foo/foo.ipynb
	posts = glob('posts/*/*.ipynb')

	# get link targets for each blog symlink
	link_targets = [os.readlink(link) for link in blog_links]

	# get list of posts which don't have symlinks yet
	new_posts = [post for post in posts if post not in link_targets]

	# sort posts by date
	new_posts_by_date = sorted(new_posts, key=quarto_post_date)

	# get the number of the last post
	n_posts = int(re.match(r'^\d+', basename(blog_links[-1])).group(0)) if blog_links else 0

	if debug:
		show_locals(exclude=_params)

	# create a numbered symlink for each post which doesn't have one yet
	for post in new_posts_by_date:
		n_posts += 1
		post_number = str(n_posts).zfill(digits)
		link_name = f'{post_number}_{os.path.basename(post)}'
		if not dry_run:
			os.symlink(post, link_name)
		if verbose:
			print(f'created symlink {link_name} -> {post}')

def update_links(blog_dir, *args, **kwargs):
	"""Update blog symlinks, wrapper to cd into blog_dir and show errors."""
	try:
		with cd(blog_dir):
			return update_links_in_cwd(*args, **kwargs)
	except Exception as e:
		if kwargs.get('debug'):
			raise
		else:
			eprint(f'error: {e}')

def main():
	"""Main function, handles command line arguments."""
	parser = argparse.ArgumentParser(description='Update blog symlinks') #, formatter_class=argparse.ArgumentDefaultsHelpFormatter, argument_default=argparse.SUPPRESS)
	default_blog_dir = env.get("BLOG_DIR", ".")
	parser.add_argument('--dir', type=str, help='blog directory', default=default_blog_dir, nargs='?')
	parser.add_argument('--digits', '-w', type=int, default=3, help='pad numbering to digits')
	parser.add_argument('--dry-run', '-n', action='store_true', help='dry run')
	parser.add_argument('--verbose', '-v', action='store_true', help='verbose')
	parser.add_argument('--debug', '-d', action='store_true', help='debug')
	args = parser.parse_args()
	update_links(args.dir, digits=args.digits, dry_run=args.dry_run, verbose=args.verbose, debug=args.debug)

if __name__ == '__main__' and not is_notebook():
	main()

ImportError: cannot import name 'is_notebook' from 'lib' (/home/sam/ai/lib/lib.py)

## Library code

### lib.py — miscellaneous functions

### Detect if running in a notebook

I don't want to run a script's main function from a notebook. 

In [ ]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

## scratch

In [ ]:
get_ipython().__class__.__name__

'ZMQInteractiveShell'

In [ ]:
is_notebook()

True